### 中醫醫案症狀詞抽取
- model: CRF
- 主症與兼症合併
- 詞位標注: OSBIE
- 移除671個詞沒有完全對應的資料點
- 以句點為分隔拆句子
- postag使用nltk

In [1]:
import json
import nltk
import numpy as np
from sklearn.metrics import classification_report

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1

In [3]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-S"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-S"
        elif i == (ne_idx + ne_len - 1):
            return "E-S"
            
    return "O"

In [4]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2].replace(" ","")
    sentences = case_content.split("。")
    
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    #print(summary_str)
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    t += len(main_list)
       
    ne_index_list_0 = []
    for ne in main_list:
        ne_idx = case_content.find(ne)
        if ne_idx != -1 :
            ne_index_list_0.append((ne_idx,len(ne)))
    
    if len(main_list) != len(ne_index_list_0):
        continue
        
    for sent in sentences:
        ne_index_list = []
        for ne in main_list:
            ne_idx = sent.find(ne)
            if ne_idx != -1 :
                ne_index_list.append((ne_idx,len(ne)))

        texts = []
        for i,w in enumerate(sent):
            label = getLabelWithWordIndex(i,ne_index_list)
            texts.append((w,label))

        docs.append(texts)

print(len(docs))

33752


In [5]:
### 計算平均句長
total_wc = 0
for doc in docs:
    total_wc += len(doc)
    
ave = total_wc / 48901
print(ave)

22.303674771477066


In [6]:
data = []

for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    tagged = nltk.pos_tag(tokens)
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

In [13]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word=' + word,
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]
        
    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word=' + word1,
            '-1.word.isdigit=%s' % word.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word=' + word1,
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [14]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [15]:
print(len(X_train))
print(len(X_test))

27001
6751


In [16]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 16526
Seconds required: 1.046

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 281108.920377
Feature norm: 1.000000
Error norm: 311285.038460
Active features: 16442
Line search trials: 1
Line search step: 0.000001
Seconds required for this iteration: 0.498

***** Iteration #2 *****
Loss: 223973.851502
Feature norm: 1.246361
Error norm: 131095.472831
Active features: 15503
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.256

***** Iteration #3 *****
Loss: 203714.440390
Feature norm: 1.420459
Error norm: 64657.493851
Active features: 15224
Line search trials: 1
Line search step: 1.000000
Seconds require

***** Iteration #45 *****
Loss: 28434.958512
Feature norm: 46.462600
Error norm: 6939.714608
Active features: 12995
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.261

***** Iteration #46 *****
Loss: 27936.260203
Feature norm: 49.310184
Error norm: 2459.873430
Active features: 13003
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.255

***** Iteration #47 *****
Loss: 27520.993062
Feature norm: 52.940484
Error norm: 5103.326536
Active features: 12967
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.254

***** Iteration #48 *****
Loss: 27330.997025
Feature norm: 53.180799
Error norm: 2818.516026
Active features: 12969
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.256

***** Iteration #49 *****
Loss: 27048.545059
Feature norm: 54.800040
Error norm: 4875.186459
Active features: 12943
Line search trials: 1
Line search step: 1.000000

***** Iteration #87 *****
Loss: 23225.024684
Feature norm: 108.112939
Error norm: 2565.247435
Active features: 11653
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.263

***** Iteration #88 *****
Loss: 23208.197077
Feature norm: 108.680614
Error norm: 3160.436513
Active features: 11632
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.255

***** Iteration #89 *****
Loss: 23194.462264
Feature norm: 109.590890
Error norm: 4183.842502
Active features: 11627
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.253

***** Iteration #90 *****
Loss: 23173.083562
Feature norm: 110.080029
Error norm: 2207.839134
Active features: 11607
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.254

***** Iteration #91 *****
Loss: 23157.476404
Feature norm: 110.575007
Error norm: 2388.554467
Active features: 11607
Line search trials: 1
Line search step: 1.0

***** Iteration #129 *****
Loss: 22856.230832
Feature norm: 124.113894
Error norm: 1912.334936
Active features: 11379
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.254

***** Iteration #130 *****
Loss: 22851.464833
Feature norm: 124.295614
Error norm: 1316.588564
Active features: 11378
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.254

***** Iteration #131 *****
Loss: 22847.900233
Feature norm: 124.540071
Error norm: 1807.516569
Active features: 11367
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.255

***** Iteration #132 *****
Loss: 22843.064242
Feature norm: 124.715578
Error norm: 1223.116064
Active features: 11359
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.253

***** Iteration #133 *****
Loss: 22839.816165
Feature norm: 124.962351
Error norm: 1949.177697
Active features: 11360
Line search trials: 1
Line search step

***** Iteration #170 *****
Loss: 22649.178236
Feature norm: 132.455028
Error norm: 1281.341456
Active features: 11248
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.259

***** Iteration #171 *****
Loss: 22645.529854
Feature norm: 132.646554
Error norm: 1913.312905
Active features: 11243
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.256

***** Iteration #172 *****
Loss: 22640.232047
Feature norm: 132.774570
Error norm: 1434.845065
Active features: 11244
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.259

***** Iteration #173 *****
Loss: 22636.773428
Feature norm: 132.958355
Error norm: 1978.327839
Active features: 11226
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.253

***** Iteration #174 *****
Loss: 22631.285907
Feature norm: 133.066994
Error norm: 1359.803793
Active features: 11231
Line search trials: 1
Line search step

In [17]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')

y_pred_train = [tagger.tag(xseq) for xseq in X_train]
y_pred_test = [tagger.tag(xseq) for xseq in X_test]

In [18]:
# Create a mapping of labels to indices
labels = {"O": 0,"B-S": 1,"I-S": 2,"E-S": 3}

# training data F1-score
predictions_train = np.array([labels[tag] for row in y_pred_train for tag in row])
truths_train = np.array([labels[tag] for row in y_train for tag in row])
print(classification_report(
    truths_train, predictions_train,
    target_names=["O", "B-SYM","I-SYM","E-SYM"]))

# test data F1-score
predictions_test = np.array([labels[tag] for row in y_pred_test for tag in row])
truths_test = np.array([labels[tag] for row in y_test for tag in row])
print(classification_report(
    truths_test, predictions_test,
    target_names=["O", "B-SYM","I-SYM","E-SYM"]))

             precision    recall  f1-score   support

          O       0.98      0.99      0.98    831628
      B-SYM       0.67      0.53      0.59     12592
      I-SYM       0.66      0.48      0.55     14923
      E-SYM       0.69      0.54      0.60     12519

avg / total       0.96      0.97      0.96    871662

             precision    recall  f1-score   support

          O       0.97      0.99      0.98    208765
      B-SYM       0.60      0.47      0.53      3216
      I-SYM       0.51      0.37      0.43      3826
      E-SYM       0.61      0.47      0.53      3203

avg / total       0.95      0.96      0.96    219010

